MIC es una empresa que se dedica a la fabricación de productos farmacéuticos. La dirección de MIC ha identificado que el costo del manejo de inventario de materias primas representa un porcentaje significativo de los costos operacionales de la compañía, por lo que está interesada en caracterizar el movimiento de este inventario en su almacén. En particular, se ha notado que el propilenglicol es uno de los productos relacionados con el mayor costo.

La política actual de manejo de inventario de MIC dicta que al final de la semana se revisa cuántos tambores de propilenglicol hay en el almacén, y si este número es menor a $m$, se hace un pedido del número de tambores necesarios para completar $k$ unidades. Los tambores pedidos están disponibles al inicio de la siguiente semana.

Además, se sabe que por errores en el manejo de los tambores, en ocasiones estos se contaminan, por lo que no se pueden usar para la producción y deben ser desechados. Así, si al inicio de la semana hay $i$ tambores en inventario, el número de tambores que son desechados durante la semana sigue una distribución uniforme sobre $\{0,1,\dots,i\}$, es decir,
$$
P[B_i = n] = \frac{1}{i+1}, \quad n=0,1,\dots,i.
$$
Finalmente, el número de tambores de propilenglicol que se usan en la producción de una semana sigue una distribución de Poisson con tasa $\lambda$ tambores/semana:
$$
P[D=n] = \frac{\lambda^n e^{-\lambda}}{n!}, \quad n=0,1,2,\dots
$$

Sea
$$
X_n = \text{Número de tambores de propilenglicol al final de la } n\text{-ésima semana},
$$
con espacio de estados
$$
S_X = \{0,1,\dots,k\}.
$$

Las probabilidades de transición están definidas de la siguiente forma:
$$
p_{i,j} = \begin{cases}
\displaystyle \sum_{n=0}^{k-j} P[D = k-j-n] \cdot P[B_i = n], & \quad 0<j,\quad i < m, \\[1mm]
\displaystyle \sum_{n=0}^{k} P[D \geq k-n] \cdot P[B_i = n], & \quad j=0,\quad i < m, \\[1mm]
\displaystyle \sum_{n=0}^{i-j} P[D = i-j-n] \cdot P[B_i = n], & \quad 0<j,\quad i \ge m, \\[1mm]
\displaystyle \sum_{n=0}^{i} P[D \geq i-n] \cdot P[B_i = n], & \quad j=0,\quad i \ge m, \\[1mm]
0, & \text{en otro caso (d.l.c.)}.
\end{cases}
$$

**Parámetros del modelo:**  
Se tomarán los valores:
$$
m=20,\quad k=50,\quad \lambda=15.
$$

Construya la matriz de transición $P$ de la cadena de Markov, luego cree la cadena de Markov utilizando la librería `jmarkov`.



In [8]:
import numpy as np
import math
from jmarkov.dtmc import dtmc

# Parámetros del modelo
m = 20            # Si el inventario es menor a m, se ordena para llegar a k
k = 50            # Inventario objetivo (se repone hasta k) cuando i < m
lam = 15          # Tasa de la demanda (Poisson) en tambores/semana

# Funciones para distribuciones

def uniforme (i,n):
    if n > i or n < 0:
        return 0
    else:
        return 1/(i+1)
    
def poisson (n):
    return ((lam**n * math.exp(-lam))/(math.factorial(n)))

def poisson_tail (n):
    s = sum(poisson(i) for i in range(n))
    return 1 - s

estados = np.array([i for i in range(0,k+1)])

matriz_transicion = np.zeros((len(estados),len(estados)))

for i in range(len(estados)):
    for j in range(len(estados)):
        if 0 < j and i < m:
            for n in range(0,k-j+1):
                matriz_transicion[i][j] += poisson(k-j-n) * uniforme(i,n)
        elif j == 0 and i < m:
            for n in range(0,k+1):
                matriz_transicion[i][j] += poisson_tail(k-n)*uniforme(i,n)
        elif 0 < j and i >= m:
            for n in range(0,i-j+1):
                matriz_transicion[i][j] += poisson(i-j-n) * uniforme(i,n)
        elif j == 0 and i >= m:
            for n in range (0,i+1):
                matriz_transicion[i][j] += poisson_tail(i-n) * uniforme(i,n)
        else:
            matriz_transicion[i][j] = 0
            
matriz_transicion

array([[9.05719943e-13, 2.13771398e-12, 6.98319900e-12, ...,
        3.44140111e-05, 4.58853481e-06, 3.05902321e-07],
       [1.97458716e-12, 4.56045649e-12, 1.46647179e-11, ...,
        1.95012729e-05, 2.44721856e-06, 1.52951160e-07],
       [4.65860683e-12, 1.04890499e-11, 3.31158815e-11, ...,
        1.31028161e-05, 1.63147904e-06, 1.01967440e-07],
       ...,
       [3.26530612e-01, 2.04081633e-02, 2.04081633e-02, ...,
        6.24290450e-09, 0.00000000e+00, 0.00000000e+00],
       [3.20000000e-01, 2.00000000e-02, 2.00000000e-02, ...,
        9.78887426e-08, 6.11804641e-09, 0.00000000e+00],
       [3.13725490e-01, 1.96078431e-02, 1.96078431e-02, ...,
        7.70753886e-07, 9.59693555e-08, 5.99808472e-09]])

In [9]:
cadena = dtmc(matriz_transicion, estados)